[Rosalind - Inferring mRNA from Protein](https://rosalind.info/problems/mrna/)

### Enunciado

For positive integers a and n, a **modulo** n (written amodn in shorthand) is the remainder when a is divided by n. For example, $29 \mod 11 = 7$ because $29 = 11×2+7$

**Modular arithmetic** is the study of addition, subtraction, multiplication, and division with respect to the modulo operation. We say that a and b are congruent modulo n if $a \mod n = b \mod n$; in this case, we use the notation $a \equiv b \mod n$

Two useful facts in modular arithmetic are that if $a \equiv b \mod n$ and c≡dmodn, then $a + c \equiv b + d \mod n$ and $a × c \equiv b × d \mod n$. To check your understanding of these rules, you may wish to verify these relationships for $a = 29$, $b = 73$, $c = 10$, $d = 32$, and $n = 11$

As you will see in this exercise, some Rosalind problems will ask for a (very large) integer solution modulo a smaller number to avoid the computational pitfalls that arise with storing such large numbers.

<font color="green">Given</font>: A protein string of length at most 1000 aa.

<font color="green">Return</font>: The total number of different RNA strings from which the protein could have been translated, modulo 1,000,000. (Don't neglect the importance of the stop codon in protein translation.)

### Sample Dataset
```
MA
```
### Sample Output
```
12
```

Primeiramente, para o nosso problema, vamos utilizar uma biblioteca de I/O para formato específico (nesse caso, CSV) e uma biblioteca para um mínimo de _data wrangling_. As operações de manipulação do _dataset_ serão necessárias para obtermos um dicionário explicitando a quantidade de _codons_ possíveis para cada um dos 20 aminoácidos.

In [ ]:
using CSV
using DataFramesMeta

In [ ]:
path = pwd() * "/codonTable.csv"

Especificamos um _path_ com nome do diretório referenciado no qual o notebook está sendo executando e fornecemos um nome de arquivo como "codonTable.csv". Esse arquivo conterá duas colunas (_codon_ e _aa_) contendo informação do códon lido e do seu respectivo aminoácido traduzido. A tabela que manipularemos para obter esses dados encontra-se no [link](https://rosalind.info/glossary/rna-codon-table/) para tabela de códons do Rosalind. A mesma se apresenta como segue:

```
UUU F      CUU L      AUU I      GUU V
UUC F      CUC L      AUC I      GUC V
UUA L      CUA L      AUA I      GUA V
UUG L      CUG L      AUG M      GUG V
UCU S      CCU P      ACU T      GCU A
UCC S      CCC P      ACC T      GCC A
UCA S      CCA P      ACA T      GCA A
UCG S      CCG P      ACG T      GCG A
UAU Y      CAU H      AAU N      GAU D
UAC Y      CAC H      AAC N      GAC D
UAA Stop   CAA Q      AAA K      GAA E
UAG Stop   CAG Q      AAG K      GAG E
UGU C      CGU R      AGU S      GGU G
UGC C      CGC R      AGC S      GGC G
UGA Stop   CGA R      AGA R      GGA G
UGG W      CGG R      AGG R      GGG G
```

In [ ]:
awkCommand = `awk '{for(i=1; i < NF; i += 2){print $i","$(i+1)}}' codons.tsv`

Para uma manipulação um pouco mais direta da tabela apresentada anteriormente, vamos utilizar a linguagem ```awk``` e nos aproveitarmos da utilidade de _Objetos de Comando_ que o Julia fornece. O artifício nos evita de utilizar expressões regulares para separação das colunas e operações de _data wrangling_ adicionais a essa tabela após a coerção para DataFrame.

O algoritmo do comando é dado por:
1. Apresente (_print_) as colunas ```i``` e ```i + 1```, separados por ",";
2. Repita (1) de ```i = 1``` a ```i < NF``` ("NF" referencia a coluna final em ```awk```) atualizando o valor pela operação ```i += 2```;
3. Aplique (1) e (2) ao arquivo codon.tsv (arquivo _tsv_ contendo a tabela anteriormente apresentada).

É válido, portanto, salvar a tabela de códons do quadro anterior em um arquivo com o nome "codons.tsv" no seu diretório de execução para que o comando funcione apropriadamente.

In [ ]:
open(path, "w") do io
    write(io, read(awkCommand, String))
end

Aqui, relizamos as operações de I/O do Julia para criação do arquivo _csv_ que utilizaremos posteriormente.

1. A função ```open``` cria um objeto para escrita de arquivo no sistema (nesse caso, especificamos o argumento ```"w"``` para que possamos inserir conteúdo no mesmo) especificando o ```path``` que definimos anteriormente;
    + Dado que o arquivo ainda não existe, a função open irá criá-lo no diretório da variável ```path``` definida (no caso, o diretório em que estamos executando o _notebook_);
2. Alocamos o objeto do I/O em uma variável local (```io```) à operação ```do``` que executamos;
3. Realizamos a escrita ao arquivo com a função ```write```, declarando a informação a ser escrita como o ```stdout``` do comando ```awkCommand```;
    + A função ```read``` ralizará a leitura do ```stdout``` e realizará um _parsing_ dessa informação ao tipo ```String``` do Julia;
4. Finalizamos a operação ```do``` com o operador ```end```, o que realizará a operação ```close``` da nossa variável ```io```.

In [ ]:
codonTable = CSV.read(path, DataFrame; header = ["codon", "aa"])

Realizamos a leitura do arquivo _csv_ criado com a biblioteca CSV. Especificamos o tipo de saída, para o arquivo lido, no segundo argumento da função ```read``` (nesse caso ```DataFrame```). O argumento opcional ```header``` especificará o nome das colunas do nosso ```DataFrame```.

In [ ]:
aaCount = @chain codonTable begin
    groupby(:aa)
    combine(nrow => :count)
    Pair.(_.aa, _.count)
    Dict
end

Aqui, realizamos as operações de _data wrangling_ necessárias à criação da estrutura de dados (dicionário) que utilizaremos. Utilizamos a _macro_ do pacote Chains.jl para criar um _pipeline_ de operações. O pacote é anexado ao _script_ pela biblioteca ```DataFramesMeta``` (assim como a biblioteca ```DataFrames```).

As opreações são como seguem:
1. Agrupamos (```groupby```) o DataFrame pela variável contendo o código dos aminoácidos;
2. Sumarizamos (```combine```) o DataFrame contando as linhas para cada um dos grupos (no caso, para cada um dos aminoácidos) utilizando ```nrow```. A coluna receberá o nome de ```count```;
3. Criamos um mapeamento (```Pair.```) de cada elemento dos aminoácidos (```aa```) a sua respectiva contagem (```count```);
    + Perceba que aqui foi executada uma função (```Pair```) com _broadcasting_ (```Pair.```) o que nos permite iterar em cada umas das linhas das colunas ```aa``` e ```count```;
    + Na linha ```Pair.(_.aa, _.count)```, o identificador ```_``` é utilizado para referenciair o nome ```codonTable``` onde os ```_``` estão posicionados. O princípio é semelhante em relação ao ```.``` na biblioteca ```dplyr``` do _superset_ do ```tidyverse``` em R;
4. Por fim, realizamos uma coerção dos mapeamentos a um dicionário utilizando um construtor ```Dict``` e finalizamos o encademanto de operações com o operador ```end```.

In [ ]:
io = open(pwd() * "/rosalind_mrna.txt", "r")
mRNA = strip(read(io, String))
close(io);

O arquivo "rosalind_mrna.txt" é obtido à partir do momento em que clicamos no botão de submissão de resposta para o problema. Baixe-o para o diretório em que irá executar o _notebook_ para que o mesmo funcione apropriadamente.

Diferente de como procedemos na outra célula de operações com I/O, para que possamos declarar uma variável ```mRNA``` que seja de definida em escopo global, salvamos o objeto de I/O em uma variável global e salvamos seu conteúdo codificado como ```String``` para uma variável global (```mRNA```). Ao mesmo tempo, realizamos o ```strip``` na _string_ alocada para que possamos _trimmar_ os _leading_ e _trailing characters_ (caracteres especiais que eventualmente aparecem no começo e fim de leitura de arquivos, como **"\n"**, **"\t"** ou **" "**).Finalmente, realizamos a operação ```close``` no objeto ```io``` previamente definido.

In [ ]:
n_π = big(1)
for i in mRNA
    n_π = (n_\pi * aaCount[string(i)]) % 1_000_000
end
n_π *= aaCount["Stop"]

n_π % 1_000_000

Aqui, realizamos a análise combinatória das possibilidades de mRNA a partir da leitura da estrutura primária de uma proteína.

1. Definimos o número de possibilidades (```n_π```) como um número do tipo ```BigInt```. A declaração dessa variável com o seguinte tipo é necessária dada a ocorrênca de _overflow_ numérico no produtório realizado dentro do laço de repetição;
2. Iteramos em cada um dos caracteres da _string_ contendo a informação da proteína, adquirindo o número de possibilidades a partir do dicionário com o número de possibilidades para cada aminoácido;
   + Dado que o dicionário (```aaCount```) criado é do tipo ```Dict{String, Int64}``` e cada dígito na _string_ ```mRNA``` é codificado como ```Char```, é necessário realizar o _casting_ de ```Char``` para ```String``` pelo construto ```string``` (com s minúsculo).
   + Como a contagem do número de possibilidades é um problema do tipo **amostragem com reposição**, utilizamos o princípio multiplicativo para realizar a análise combinatória do problema
     - Para isso, precisamos apenas atualizar a variável ```n_π``` multiplicando-a pelo número de possibilidades de cada aminoácido, enquanto percorremos a string ```mRNA```, como na linha aninhada no laço de repetição
3. Após finalizar o _for loop_, temos de considerar o número de possibilidades do códon de parada, então atualizamos a variável ```n_π``` multiplicando pelo número de possibilidades do ```"Stop"```

Finalmente, o problema nos pede para obter o resultado do de $n_\pi \mod 10^6$, sendo $n_\pi$ o número de possibilidades de mRNA para a proteína em questão. Realizamos então a operação módulo (```%```) com os operandos ```n_π``` e ```1_000_000```, obtendo nosso resultado.

Um modo de contornar a necessidade de _type cast_ de cada um dos caracteres da _string_ ```mRNA``` poderia ser realizado da seguinte forma:

```
io = open(pwd() * "/rosalind_mrna.txt", "r")
mRNA = read(io, String)
close(io);

mRNArray = split(mRNA, "")
mRNArray[end] = "Stop"

n_π = big(1)
for i in mRNArrray
    n_π *= aaCount[i]
end

n_π % 1_000_000
```

Ou, mais diretamente, especificariamos o tipo do dicionário para ```Dict{Char, Int64}```

Para a possibilidade de _overflow_ numérico, poderiamos realizar um desvio condicional que testasse a condição de ```n_π > 1e6``` e, caso verdadeiro, realizariamos a operação módulo e alocaríamos o valor em um _array_. Finalmente, realizariamos o produtório dos valores do _array_ considerando a propriedade multiplicatica do módulo.